# Test with 10 MI features and 30 MU points

In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
 

In [53]:
# Load dataset
df = pd.read_csv("../name_of_file.csv")

# Separate features and labels
X = df.drop(columns=['loan_status', 'confidence']).values
y = df['loan_status'].values

print(df['loan_status'].value_counts(normalize=True))

random_number = np.random.randint(0, 1000)

# Split by percentage
train_percentage = 0.8  # 80% train, 20% test
X_train, X_test, train_labels, test_labels = train_test_split(
    X, y, train_size=train_percentage, random_state=random_number, shuffle=True
)


# Step 3: create scaler
scaler = MinMaxScaler(feature_range=(0, np.pi))

# Step 4: fit scaler on your training set or just transform this vector
X_train = scaler.fit_transform(X_train.astype(float))  # or scaler.transform(x_array) if already fitted
X_test = scaler.fit_transform(X_test.astype(float))

X_train = [row.tolist() for row in X_train]
X_test = [row.tolist() for row in X_test]
train_labels = [row.tolist() for row in train_labels]
test_labels = [row.tolist() for row in test_labels]



loan_status
0    0.5
1    0.5
Name: proportion, dtype: float64


In [54]:

# Actual sizes after splitting
train_size = len(X_train)
test_size = len(X_test)


In [55]:
# Empty kernel matrix
num_samples = np.shape(X_train)[0]
kernel_matrix = np.full((num_samples, num_samples), np.nan)
test_matrix = np.full((test_size, num_samples), np.nan)

In [56]:
# from qiskit.circuit.library import zz_feature_map
# fm = zz_feature_map(feature_dimension=np.shape(train_data)[1], entanglement='linear', reps=1)
 
from qiskit.circuit.library import zz_feature_map

fm = zz_feature_map(feature_dimension=np.shape(X_train)[1],  entanglement="linear", reps=2)

In [57]:
from qiskit.primitives import StatevectorSampler
from qiskit.circuit.library import unitary_overlap
 

num_shots = 300
 
# Evaluate the problem using state vector-based primitives from Qiskit.
sampler = StatevectorSampler()
 
for x1 in range(0, train_size):
    for x2 in range(x1 + 1, train_size):
        unitary1 = fm.assign_parameters(X_train[x1])
        unitary2 = fm.assign_parameters(X_train[x2])
 
        # Create the overlap circuit
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()

        print("circuit depth = ", overlap_circ.decompose().depth())
        #overlap_circ.decompose().draw("mpl", scale=0.6, style="iqp")
        print(
            "two-qubit depth",
            overlap_circ.decompose().depth(lambda instr: len(instr.qubits) > 1),
        )
    
        # These lines run the qiskit sampler primitive.
        counts = (
            sampler.run([overlap_circ], shots=num_shots)
            .result()[0]
            .data.meas.get_int_counts()
        )
 
        # Assign the probability of the 0 state to the kernel matrix, and the transposed element (since this is an inner product)
        kernel_matrix[x1, x2] = counts.get(0, 0.0) / num_shots
        kernel_matrix[x2, x1] = counts.get(0, 0.0) / num_shots
    # Fill in on-diagonal elements with 1, again, since this is an inner-product corresponding to probability (or alter the code to check these entries and verify they yield 1)
    kernel_matrix[x1, x1] = 1
 
print("training done")


circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-qubit depth 45
circuit depth =  75
two-q

In [58]:

# Similar process to above, but for testing data.
for x1 in range(0, test_size):
    for x2 in range(0, train_size):
        unitary1 = fm.assign_parameters(X_test[x1])
        unitary2 = fm.assign_parameters(X_train[x2])
 
        # Create the overlap circuit
        overlap_circ = unitary_overlap(unitary1, unitary2)
        overlap_circ.measure_all()
 
        counts = (
            sampler.run([overlap_circ], shots=num_shots)
            .result()[0]
            .data.meas.get_int_counts()
        )
 
        test_matrix[x1, x2] = counts.get(0, 0.0) / num_shots
 
print("test matrix done")

test matrix done


In [59]:
print(kernel_matrix)

[[1.         0.         0.         ... 0.00333333 0.01666667 0.        ]
 [0.         1.         0.00333333 ... 0.         0.         0.        ]
 [0.         0.00333333 1.         ... 0.00666667 0.         0.01333333]
 ...
 [0.00333333 0.         0.00666667 ... 1.         0.         0.        ]
 [0.01666667 0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.01333333 ... 0.         0.         1.        ]]


In [60]:
print(test_matrix)

[[0.         0.         0.00333333 ... 0.         0.00666667 0.        ]
 [0.         0.03666667 0.         ... 0.00333333 0.         0.        ]
 [0.         0.         0.01       ... 0.         0.         0.        ]
 ...
 [0.         0.00666667 0.00333333 ... 0.         0.         0.        ]
 [0.         0.00333333 0.         ... 0.         0.         0.01333333]
 [0.00333333 0.         0.         ... 0.         0.         0.        ]]


In [61]:
# Save as NumPy files
np.save("kernel_matrix_sim.npy", kernel_matrix)
np.save("test_matrix_sim.npy", test_matrix)

In [62]:
# import a support vector classifier from a classical ML package.
from sklearn.svm import SVC
 
# Specify that you want to use a pre-computed kernel matrix
qml_svc = SVC(kernel="precomputed")
# Feed in the pre-computed matrix and the labels of the training data. The classical algorithm gives you a fit.
qml_svc.fit(kernel_matrix, train_labels)

# Now use the .score to test your data, using the matrix of test data, and test labels as your inputs.
qml_score_precomputed_kernel = qml_svc.score(test_matrix, test_labels)
print(f"Precomputed kernel classification test score: {qml_score_precomputed_kernel}")

Precomputed kernel classification test score: 0.45614035087719296


### Classical


In [63]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

# Classical linear SVM
linear_svc = SVC(kernel='linear', C=1.0)

# Train on your training data
linear_svc.fit(X_train, train_labels)

# Predict on test data
y_pred = linear_svc.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("Linear SVM accuracy:", accuracy)


Linear SVM accuracy: 0.3508771929824561


In [64]:
linear_svc = SVC(kernel='rbf', C=1.0)

# Train on your training data
linear_svc.fit(X_train, train_labels)

# Predict on test data
y_pred = linear_svc.predict(X_test)

# Evaluate accuracy
accuracy = accuracy_score(test_labels, y_pred)
print("RBF SVM accuracy:", accuracy)


RBF SVM accuracy: 0.5614035087719298
